In [1]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

In [49]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, GRU

In [24]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from sklearn.metrics import f1_score

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!unzip /content/drive/My\ Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip

Archive:  /content/drive/My Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
  inflating: test_labels.csv         


In [10]:
!unzip /content/drive/My\ Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/test.csv.zip

Archive:  /content/drive/My Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/test.csv.zip
  inflating: test.csv                


In [11]:
!unzip /content/drive/My\ Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/train.csv.zip 

Archive:  /content/drive/My Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/train.csv.zip
  inflating: train.csv               


In [12]:
!ls /content/drive/MyDrive/Kaggle/jigsaw-toxic-comment-classification-challenge

sample_submission.csv.zip  test.csv.zip  test_labels.csv.zip  train.csv.zip


In [13]:
!realpath test.csv

/content/test.csv


In [17]:
training_data = pd.read_csv("/content/train.csv")
testing_data = pd.read_csv("/content/test.csv")
testing_labels = pd.read_csv("/content/test_labels.csv")

In [15]:
drive.flush_and_unmount()

In [16]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2022-06-02 00:26:33--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-06-02 00:26:33--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-02 00:26:33--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [18]:
!ls

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip  test.csv	     train.csv
glove.6B.200d.txt  glove.6B.50d.txt   sample_data   test_labels.csv


In [19]:
# EMBEDDING_FILE = "glove.6B.50d.txt"
embedding = "glove.6B.50d.txt"

In [35]:
# size of embedding vector (total nb of words)
max_len_embedding = 20000
# size of each word
size = 50
# maximum length of a comment (in words)
max_len = 100

In [21]:
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
nb_labels = len(labels)

In [25]:
# Preprocessing

# Pattern form filtering english stopwords, taken from https://stackoverflow.com/questions/19560498/faster-way-to-remove-stop-words-in-python
stopword_pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')

def preprocess_data(text):

  # Removing trailing spaces and leading spaces
  text = text.strip()
  # Lowercase
  text = text.lower()
  # Remove numbers
  text = re.sub(r'[0-9]', '', text)
  # Remove stopwords
  text = stopword_pattern.sub('', text)
  # Remove special characters
  text = re.sub(r'[^\w\s]', '', text)
  # Remove consecutive spaces
  text = re.sub(r' +', ' ', text)
  # Remove new lines
  text = re.sub(r'\n', ' ', text)

  return text

In [28]:
training_data.comment_text = training_data["comment_text"].map(preprocess_data)

In [30]:
train_y = training_data[labels].fillna("_na_").values
testing_data.comment_text = testing_data["comment_text"].map(preprocess_data)

In [33]:
testing_clean = pd.merge(testing_data, testing_labels)
testing_clean = testing_clean.drop(testing_clean.index[testing_clean['toxic'] == -1])
train_list = training_data['comment_text'].tolist()
test_list = testing_clean['comment_text'].tolist()
len(test_list)

63978

In [36]:
tokenizer = Tokenizer(num_words=max_len_embedding)

In [37]:
tokenizer = Tokenizer(num_words=max_len_embedding)
tokenizer.fit_on_texts(train_list)
list_tokenized_train = tokenizer.texts_to_sequences(train_list)
list_tokenized_test = tokenizer.texts_to_sequences(test_list)

train_x = pad_sequences(list_tokenized_train, maxlen=max_len)
test_x = pad_sequences(list_tokenized_test, maxlen=max_len)

In [38]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

embeddings_dict = dict(get_coefs(*elt.strip().split()) for elt in open(embedding))

In [39]:
embedding_stack = np.stack(embeddings_dict.values())

mean = embedding_stack.mean()
std = embedding_stack.std()
print(mean)
std

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


0.020940498


0.6441043

In [40]:
index = tokenizer.word_index
total_words = min(max_len_embedding, len(index))
embedding_matrix = np.random.normal(mean, std, (total_words, size))
for word, i in index.items():
    if i >= max_len_embedding: 
      continue
    embedding_vector = index.get(word)
    if embedding_vector is not None: 
      embedding_matrix[i] = embedding_vector

In [47]:
lstm_model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(max_len,)),
  tf.keras.layers.Embedding(input_dim=max_len_embedding, output_dim=size, input_length=max_len_embedding, weights=[embedding_matrix]),
  tf.keras.layers.Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(50, activation="relu"),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(nb_labels, activation="sigmoid")
])

lstm_model.summary()

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           1000000   
                                                                 
 bidirectional (Bidirectiona  (None, 100, 100)         40400     
 l)                                                              
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 50)                5050      
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 3

In [48]:
lstm_model.fit(train_x, train_y, batch_size=32, epochs=2, validation_split=0.1);

Epoch 1/2
4488/4488 [==============================] - 3773s 839ms/step - loss: 0.1418 - accuracy: 0.9628 - val_loss: 0.1403 - val_accuracy: 0.9940
Epoch 2/2
4488/4488 [==============================] - 3555s 792ms/step - loss: 0.1380 - accuracy: 0.9938 - val_loss: 0.1385 - val_accuracy: 0.9940


In [50]:
y_test_lstm = lstm_model.predict([test_x], batch_size=1024, verbose=1)

63/63 [==============================] - 5s 66ms/step


In [71]:
f1_score_val_lstm = f1_score(np.argmax(testing_clean[labels].values, axis=1), np.argmax(y_test_lstm, axis=1), average="weighted")
f1_score_val_lstm

0.9964142608765079

In [73]:
avg = 0
for i, label in enumerate(labels):
    print(label, ":")
    pb = y_test_lstm[:, i] >= 0.5
    score = f1_score(testing_clean[label], pb, average="weighted")
    print(score)
    avg += score

avg /= nb_labels
print("Average f1-score:", avg)

toxic :
0.8595949786595427
severe_toxic :
0.9914037297776611
obscene :
0.9143192385431218
threat :
0.995055710671004
insult :
0.9203891180722282
identity_hate :
0.9833378976420114
Average f1-score: 0.9440167788942615


In [ ]:
# GRU model

In [74]:
gru_model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(max_len,)),
  tf.keras.layers.Embedding(input_dim=max_len_embedding, output_dim=size, input_length=max_len_embedding, weights=[embedding_matrix]),
  tf.keras.layers.SpatialDropout1D(0.3),
  tf.keras.layers.Bidirectional(GRU(70, return_sequences=True)),  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(50, activation="relu"),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(nb_labels, activation="sigmoid")
])

gru_model.summary()

gru_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 50)           1000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 100, 50)          0         
 ropout1D)                                                       
                                                                 
 bidirectional_1 (Bidirectio  (None, 100, 140)         51240     
 nal)                                                            
                                                                 
 global_average_pooling1d_1   (None, 140)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 50)                7050      
                                                      

In [75]:
gru_model.fit(train_x, train_y, batch_size=32, epochs=2, validation_split=0.1)

Epoch 1/2
4488/4488 [==============================] - 61s 12ms/step - loss: 0.1419 - accuracy: 0.9022 - val_loss: 0.1390 - val_accuracy: 0.9940
Epoch 2/2
4488/4488 [==============================] - 57s 13ms/step - loss: 0.1376 - accuracy: 0.9930 - val_loss: 0.1370 - val_accuracy: 0.9940


In [76]:
y_test_gru = gru_model.predict([test_x], batch_size=1024, verbose=1)

63/63 [==============================] - 1s 14ms/step


In [78]:
f1_score_val_gru = f1_score(np.argmax(testing_clean[labels].values, axis=1), np.argmax(y_test_gru, axis=1), average="weighted")
f1_score_val_gru

0.9964142608765079

In [79]:
avg = 0
for i, label in enumerate(labels):
    print(label, ":")
    pb = y_test_gru[:, i] >= 0.5
    score = f1_score(testing_clean[label], pb, average="weighted")
    print(score)
    avg += score

avg /= nb_labels
print("Average f1-score:", avg)

toxic :
0.8595949786595427
severe_toxic :
0.9914037297776611
obscene :
0.9143192385431218
threat :
0.995055710671004
insult :
0.9203891180722282
identity_hate :
0.9833378976420114
Average f1-score: 0.9440167788942615


In [ ]:
# GRU model is faster to train and performs just as good

In [ ]:
# subword embedding (fasttext) : trying to improve toxic obscene and insult labels f1 scores

In [81]:
fasttext_embedding_file = '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'

In [83]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [84]:
!unzip /content/drive/MyDrive/Kaggle/crawl-300d-2M.vec.zip

Archive:  /content/drive/MyDrive/Kaggle/crawl-300d-2M.vec.zip
  inflating: crawl-300d-2M.vec       
  inflating: __MACOSX/._crawl-300d-2M.vec  


In [85]:
!realpath crawl-300d-2M.vec

/content/crawl-300d-2M.vec


In [86]:
fasttext_embedding_file = "/content/crawl-300d-2M.vec"

In [87]:
fasttext_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(fasttext_embedding_file))

In [88]:
drive.flush_and_unmount()

In [89]:
fasttext_stack = np.stack(fasttext_index.values())

mean = fasttext_stack.mean()
std = fasttext_stack.std()
print(mean)
std

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


0.0055286596


0.34703913

In [90]:
index = tokenizer.word_index
total_words = min(max_len_embedding, len(index))
fasttext_embedding_matrix = np.random.normal(mean, std, (total_words, size))
for word, i in index.items():
    if i >= max_len_embedding: continue
    embedding_vector = index.get(word)
    if embedding_vector is not None: fasttext_embedding_matrix[i] = embedding_vector

In [80]:
# Reimplementing the model with the better accuracy using subword embeddings (bidirectional GRU and LSTM models implemented have the same accuracy, so I chose GRU because it was faster during training)

In [91]:
gru_model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(max_len,)),
  tf.keras.layers.Embedding(input_dim=max_len_embedding, output_dim=size, input_length=max_len_embedding, weights=[fasttext_embedding_matrix]),
  tf.keras.layers.SpatialDropout1D(0.3),
  tf.keras.layers.Bidirectional(GRU(70, return_sequences=True)),  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(50, activation="relu"),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(nb_labels, activation="sigmoid")
])

In [92]:
gru_model.summary()

gru_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 50)           1000000   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 100, 50)          0         
 lDropout1D)                                                     
                                                                 
 bidirectional_2 (Bidirectio  (None, 100, 140)         51240     
 nal)                                                            
                                                                 
 global_average_pooling1d_2   (None, 140)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 50)                7050      
                                                      

In [93]:
gru_model.fit(train_x, train_y, batch_size=32, epochs=2, validation_split=0.1)

Epoch 1/2
4488/4488 [==============================] - 59s 12ms/step - loss: 0.1414 - accuracy: 0.9064 - val_loss: 0.1416 - val_accuracy: 0.9940
Epoch 2/2
4488/4488 [==============================] - 55s 12ms/step - loss: 0.1375 - accuracy: 0.9933 - val_loss: 0.1371 - val_accuracy: 0.9940


In [94]:
y_test_gru_ft = gru_model.predict([test_x], batch_size=1024, verbose=1)

63/63 [==============================] - 2s 14ms/step


In [95]:
f1_score_val_gru_ft = f1_score(np.argmax(testing_clean[labels].values, axis=1), np.argmax(y_test_gru_ft, axis=1), average="weighted")
f1_score_val_gru_ft

0.9964142608765079

In [96]:
avg = 0
for i, label in enumerate(labels):
    print(label, ":")
    pb = y_test_gru_ft[:, i] >= 0.5
    score = f1_score(testing_clean[label], pb, average="weighted")
    print(score)
    avg += score

avg /= nb_labels
print("Average f1-score:", avg)

toxic :
0.8595949786595427
severe_toxic :
0.9914037297776611
obscene :
0.9143192385431218
threat :
0.995055710671004
insult :
0.9203891180722282
identity_hate :
0.9833378976420114
Average f1-score: 0.9440167788942615


In [ ]:
# no real improvement